# Babamul API Client

This notebook demonstrates all the capabilities of the Babamul REST API through the `babamul` Python package.

The API provides access to ZTF and LSST transient alert data, including:
- **Authentication** (signup, login, profile)
- **Kafka credentials** management
- **Alert queries** (by object ID, cone search, with photometric/quality filters)
- **Full object retrieval** with photometry history
- **Cutout images** (science, template, difference)
- **Object search** by partial ID

## 0. Setup

Copy `.env.example` to `.env` and fill in your credentials:

```
# API credentials
EMAIL=your_email@example.com
PASSWORD=your_password
```

In [ ]:
import os

import dotenv

from babamul.consumer import AlertConsumer
from babamul.api import APIClient

dotenv.load_dotenv()

## 1. Authentication

The API requires authentication for most endpoints. You can authenticate by logging in with email and password.

The `APIClient` supports context manager usage (`with` statement) for automatic cleanup.

In [ ]:
client = APIClient()

# Login with credentials
email = os.environ["EMAIL"]
password = os.environ["PASSWORD"]
token = client.login(email, password)

print(f"Authenticated: {client.is_authenticated}")

### User Profile

Once authenticated, you can retrieve your profile information.

In [ ]:
profile = client.get_profile()

print(f"Username: {profile.username}")
print(f"Email:    {profile.email}")
print(f"ID:       {profile.id}")

## 2. Kafka Credentials Management

The API allows you to create and manage Kafka credentials for streaming alerts in real-time via `AlertConsumer`.

In [ ]:
# List existing credentials
credentials = client.list_kafka_credentials()
print(f"Existing credentials: {len(credentials)}")
for cred in credentials:
    print(f"  - {cred.name} (username: {cred.kafka_username})")

In [ ]:
# Create a new credential
new_cred = client.create_kafka_credential("notebook-demo")

print(f"Name:           {new_cred.name}")
print(f"Kafka username: {new_cred.kafka_username}")
print(f"Kafka password: {new_cred.kafka_password}")

In [ ]:
# Delete the credential we just created
deleted = client.delete_kafka_credential(new_cred.id)
print(f"Deleted: {deleted}")

## 3. Object Search

Search for objects by partial ID.

In [ ]:
# Search with a partial ZTF object ID
ztf_object_id = "ZTF18abmrfqv"
partial_id = ztf_object_id[:11]  # Use first 11 characters

results = client.search_objects(partial_id, limit=5)
print(f"Sample of objects matching '{partial_id}':")
for r in results:
    print(f"  {r.objectId}  (RA={r.ra:.5f}, Dec={r.dec:.5f})")

## 4. Querying Alerts

Alerts can be queried in multiple ways:
- **By object ID**: retrieve all alerts for a specific object
- **By cone search** (RA, Dec, radius): find alerts in a region of the sky
- **With filters**: narrow results by magnitude, DRB score, or Julian Date range

### 4.1 Query ZTF alerts by object ID

In [ ]:
ztf_object_id = "ZTF18abmrfqv"

ztf_alerts = client.get_alerts("ztf", object_id=ztf_object_id)
print(f"Found {len(ztf_alerts)} ZTF alerts for {ztf_object_id}")

# Inspect the first alert
alert = ztf_alerts[0]
print(f"\nFirst alert:")
print(f"  candid:   {alert.candid}")
print(f"  objectId: {alert.objectId}")
print(f"  RA:       {alert.candidate.ra:.6f}")
print(f"  Dec:      {alert.candidate.dec:.6f}")
print(f"  magpsf:   {alert.candidate.magpsf}")
print(f"  band:     {alert.candidate.band}")
print(f"  drb:      {alert.candidate.drb}")
print(f"  jd:       {alert.candidate.jd}")

In [ ]:
# Alert properties (classification flags)
props = alert.properties
print(f"Properties:")
print(f"  rock:                 {props.rock}")
print(f"  star:                 {props.star}")
print(f"  near_brightstar:      {props.near_brightstar}")
print(f"  stationary:           {props.stationary}")

# Classifications (if available)
if alert.classifications:
    print(f"\nClassifications:")
    for name, score in alert.classifications.items():
        print(f"  {name}: {score:.4f}")

### 4.2 Query LSST alerts by object ID

In [ ]:
lsst_object_id = "LSST_OBJECT_ID"  # Replace with a valid LSST object ID

lsst_alerts = client.get_alerts("lsst", object_id=lsst_object_id)
print(f"Found {len(lsst_alerts)} LSST alerts for {lsst_object_id}")

if lsst_alerts:
    alert_lsst = lsst_alerts[0]
    print(f"\nFirst alert:")
    print(f"  candid:   {alert_lsst.candid}")
    print(f"  objectId: {alert_lsst.objectId}")
    print(f"  RA:       {alert_lsst.candidate.ra:.6f}")
    print(f"  Dec:      {alert_lsst.candidate.dec:.6f}")
    print(f"  magpsf:   {alert_lsst.candidate.magpsf}")
    print(f"  band:     {alert_lsst.candidate.band}")

### 4.3 Cone search (RA, Dec, radius)

In [ ]:
# Cone search for ZTF alerts around the first alert's position
ra = ztf_alerts[0].candidate.ra
dec = ztf_alerts[0].candidate.dec
radius_arcsec = 0.1

cone_alerts = client.get_alerts("ztf", ra=ra, dec=dec, radius_arcsec=radius_arcsec)
print(f"Found {len(cone_alerts)} ZTF alerts within {radius_arcsec} arcsec of (RA={ra:.5f}, Dec={dec:.5f})")

# Show unique objects in this region
unique_objects = set(a.objectId for a in cone_alerts)
print(f"Unique objects: {unique_objects}")

In [ ]:
# Cone search for LSST alerts around a specific position
ra=83.6331
dec=22.0145
radius_arcsec=0.1

cone_alerts = client.get_alerts("lsst", ra=ra, dec=dec, radius_arcsec=radius_arcsec)
print(f"Found {len(cone_alerts)} LSST alerts within {radius_arcsec} arcsec of (RA={ra:.5f}, Dec={dec:.5f})")

# Show unique objects in this region
unique_objects = set(a.objectId for a in cone_alerts)
print(f"Unique objects: {unique_objects}")

### 4.4 Filtering by magnitude

In [ ]:
# Get all alerts first
all_alerts = client.get_alerts("ztf", object_id=ztf_object_id)
print(f"All alerts: {len(all_alerts)}")

# Filter by magnitude range
filtered_alerts = client.get_alerts(
    "ztf",
    object_id=ztf_object_id,
    min_magpsf=18.5,
    max_magpsf=22.0,
)
print(f"Alerts with 18.0 <= magpsf <= 22.0: {len(filtered_alerts)}")

for a in filtered_alerts[:5]:
    print(f"  candid={a.candid}, magpsf={a.candidate.magpsf:.2f}, band={a.candidate.band}")

### 4.5 Filtering by DRB (Deep Real-Bogus) score

In [ ]:
# Only keep high-confidence real detections (DRB >= 0.9)
high_drb_alerts = client.get_alerts(
    "ztf",
    object_id=ztf_object_id,
    min_drb=0.9,
    max_drb=1.0,
)
print(f"All alerts: {len(all_alerts)}")
print(f"Alerts with DRB >= 0.9: {len(high_drb_alerts)}")

for a in high_drb_alerts[:5]:
    print(f"  candid={a.candid}, drb={a.candidate.drb}, magpsf={a.candidate.magpsf:.2f}")

## 5. Full Object Retrieval

While `get_alerts` returns alert summaries, `get_object` returns the complete object with:
- Full photometry history (`prv_candidates`, `prv_nondetections`, `fp_hists`)
- Cutout images
- Cross-survey matches

### 5.1 Get a full ZTF object

In [ ]:
ztf_obj = client.get_object("ztf", ztf_object_id)

print(f"Object:          {ztf_obj.objectId}")
print(f"Survey:          {ztf_obj.survey}")
print(f"candid:          {ztf_obj.candid}")
print(f"RA:              {ztf_obj.candidate.ra:.6f}")
print(f"Dec:             {ztf_obj.candidate.dec:.6f}")
print(f"DRB:             {ztf_obj.drb}")
print(f"Photometry history:")
print(f"  prv_candidates:    {len(ztf_obj.prv_candidates)}")
print(f"  prv_nondetections: {len(ztf_obj.prv_nondetections)}")
print(f"  fp_hists:          {len(ztf_obj.fp_hists)}")

In [ ]:
# You can also fetch the full object directly from an API alert
api_alert = ztf_alerts[0]
full_obj = api_alert.fetch_object(client)

print(f"Fetched the full object {full_obj.objectId}")
print(f"Total photometry points: {len(full_obj.get_photometry())}")

### 5.2 Get a full LSST object

In [ ]:
if lsst_object_id == "LSST_OBJECT_ID":
    print("Please replace 'LSST_OBJECT_ID' with a valid LSST object ID to run this section.")
    lsst_obj = None
else:
    lsst_obj = client.get_object("lsst", lsst_object_id)
    print(f"Object:          {lsst_obj.objectId}")
    print(f"Survey:          {lsst_obj.survey}")
    print(f"candid:          {lsst_obj.candid}")
    print(f"RA:              {lsst_obj.candidate.ra:.6f}")
    print(f"Dec:             {lsst_obj.candidate.dec:.6f}")
    print(f"Photometry history:")
    print(f"  prv_candidates: {len(lsst_obj.prv_candidates)}")
    print(f"  fp_hists:       {len(lsst_obj.fp_hists)}")

### 5.3 Cross-survey matches

In [ ]:
# Check if ZTF object has an LSST counterpart
if ztf_obj.survey_matches and ztf_obj.survey_matches.lsst:
    lsst_match = ztf_obj.survey_matches.lsst
    print(f"LSST match found for {ztf_obj.objectId}:")
    print(f"  LSST objectId: {lsst_match.objectId}")
    print(f"  RA: {lsst_match.ra:.6f}, Dec: {lsst_match.dec:.6f}")
    print(f"  LSST photometry points: {len(lsst_match.prv_candidates)}")
else:
    print(f"No LSST match found for {ztf_obj.objectId}")

# Check if LSST object has a ZTF counterpart
if lsst_object_id == "LSST_OBJECT_ID":
    print("Please replace 'LSST_OBJECT_ID' with a valid LSST object ID to run this section.")
elif lsst_obj.survey_matches and lsst_obj.survey_matches.ztf:
    ztf_match = lsst_obj.survey_matches.ztf
    print(f"\nZTF match found for {lsst_obj.objectId}:")
    print(f"  ZTF objectId: {ztf_match.objectId}")
    print(f"  RA: {ztf_match.ra:.6f}, Dec: {ztf_match.dec:.6f}")
else:
    print(f"\nNo ZTF match found for {lsst_obj.objectId}")

## 6. Photometry

The `get_photometry()` method combines all photometry sources (previous candidates, forced photometry, non-detections) into a single sorted list. By default, it deduplicates entries with the same (jd, band) pair.

In [ ]:
# Get combined, deduplicated photometry
photometry = ztf_obj.get_photometry(deduplicated=True)
print(f"Total photometry points (deduplicated): {len(photometry)}")

# Compare with non-deduplicated
photometry_raw = ztf_obj.get_photometry(deduplicated=False)
print(f"Total photometry points (raw):          {len(photometry_raw)}")

# Inspect a few points
print(f"\nFirst 5 photometry points:")
for p in photometry[:5]:
    mag_str = f"{p.magpsf:.2f}" if p.magpsf is not None else "non-det"
    print(f"  JD={p.jd:.4f}, band={p.band}, mag={mag_str}")

## 7. Cutout Images

Each alert has three associated cutout images:
- **Science**: the actual observation
- **Template**: the reference image
- **Difference**: science minus template

### 7.1 Fetch cutouts from the API

In [ ]:
# It is possible to fetch cutouts directly from an alert retrieved via the API
cutouts = ztf_alerts[0].fetch_cutouts(client)
print(f"Cutouts for candid {cutouts.candid} retrieved from API:")
print(f"  Science:    {len(cutouts.cutoutScience)} bytes" if cutouts.cutoutScience else "  Science:    None")
print(f"  Template:   {len(cutouts.cutoutTemplate)} bytes" if cutouts.cutoutTemplate else "  Template:   None")
print(f"  Difference: {len(cutouts.cutoutDifference)} bytes" if cutouts.cutoutDifference else "  Difference: None")

In [ ]:
# It is also possible to fetch cutouts from a Kafka alert
new_cred = client.create_kafka_credential("notebook-demo") # Create new credential for Kafka consumer
print(f"Created new credential {new_cred.id}")
ztf_consumer = AlertConsumer(
    topics="babamul.ztf.no-lsst-match.hosted",
    offset="earliest",
    username=new_cred.kafka_username,
    password=new_cred.kafka_password,
)
for alert in ztf_consumer:
    cutouts = alert.fetch_cutouts(client)
    print(f"Cutouts for candid {cutouts.candid} retrieved from Kafka alert:")
    print(f"  Science:    {len(cutouts.cutoutScience)} bytes" if cutouts.cutoutScience else "  Science:    None")
    print(f"  Template:   {len(cutouts.cutoutTemplate)} bytes" if cutouts.cutoutTemplate else "  Template:   None")
    print(f"  Difference: {len(cutouts.cutoutDifference)} bytes" if cutouts.cutoutDifference else "  Difference: None")
    break

ztf_consumer.close()
deleted = client.delete_kafka_credential(new_cred.id) # Clean up credential
print(f"Deleted credential {new_cred.id}: {deleted}")

### 7.2 Display cutout images

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from io import BytesIO
from PIL import Image

ztf_obj.plot_cutouts(orientation="horizontal", show=True)
ztf_obj.plot_cutouts(orientation="vertical", show=True)

## 8. Cleanup

In [ ]:
client.close()
print("Client closed.")